In [2]:
import pandas as pd
import re
import pymorphy2
from nltk.tokenize import word_tokenize 
from gensim.models import Word2Vec


from tqdm import tqdm
tqdm.pandas()

/home/marie/newenv/lib/python3.8/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/home/marie/newenv/lib/python3.8/site-packages/sklearn/utils/multiclass.py:13: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix
/home/marie/newenv/lib/python3.8/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `

In [3]:
df = pd.read_csv('../lab_1/rureviews/women-clothing-accessories.3-class.balanced.csv', encoding="utf8", sep='\t')

In [4]:
df = df[df['sentiment'] != 'neautral']

In [5]:
df

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative
...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive
89997,спасибо большое ) продовца рекомендую.. заказа...,positive
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive


In [6]:
# Load model directly

from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [13]:
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.2524, -0.5321,  0.4496,  ...,  1.1428, -0.6238, -0.0745],
         [ 0.5374,  0.0701,  0.4880,  ...,  1.0800, -0.5560, -0.5564],
         [ 0.4686, -0.3286,  0.4782,  ...,  1.1656, -0.7389, -0.3920],
         ...,
         [ 0.4822, -1.0822,  0.9026,  ...,  1.8029, -1.1343, -0.1034],
         [ 0.1283, -0.6550,  0.3723,  ...,  0.7817, -0.9173, -0.0401],
         [ 0.1645, -0.5239,  0.6638,  ...,  0.7964, -0.7326, -0.2642]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 0.3479, -0.0466,  0.4564, -0.2464, -0.0563,  0.5865,  0.4807,  0.2475,
         -0.5663,  0.4260,  0.0136, -0.3356, -0.3281,  0.0391,  0.3874, -0.4077,
          0.8111,  0.2171,  0.4523, -0.5184, -0.9998, -0.5465, -0.1472, -0.5278,
         -0.5858,  0.3390, -0.4160,  0.3991,  0.4012, -0.4298,  0.1618, -0.9998,
          0.7481,  0.7902,  0.3325, -0.4037,  0.1348,  0.2374,  0.2554, -0.1603,
         -0.3249,  0.0911, -0.49

In [7]:
from tqdm import tqdm
tqdm.pandas()

In [15]:
df = df[df['len_tokens'] < 512]

In [10]:
import torch

In [11]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [12]:
device

'cuda'

In [ ]:
preproc =  pd.DataFrame()
preproc["tokenized"] = df['review'].progress_apply((lambda x: tokenizer(x, return_tensors='pt')))
preproc["vectors"] = preproc["tokenized"].progress_apply(lambda x: model(**x)[0][0][0])

  0%|          | 63/59996 [00:02<44:27, 22.46it/s]

In [26]:
preproc["vectors"] = preproc["vectors"].apply(lambda x: x[0][0][0])

In [27]:
preproc["vectors"].iloc[0].shape

torch.Size([768])

In [13]:
df['len_tokens'] = df['review'].progress_apply(lambda x: len(tokenizer.tokenize(x)))

100%|██████████| 60000/60000 [00:18<00:00, 3194.75it/s]


In [14]:
df[df['len_tokens'] >= 512]

,review,sentiment,len_tokens
984,РАНЕЕ Я КУПИЛА В ЭТОМ МАГАЗИНЕ ЧЕРНУЮ ЮБКУ РАЗ...,negative,539
2402,ОТВРАТИТЕЛЬНЫЙ ПРОДАВЕЦ! НА МОИ СООБЩЕНИЯ ТУПО...,negative,530
27640,"ПЛОХОЙ ПРОДАВЕЦ. ПОКА СПОР НЕ ОТКРЫЛ, НЕ ОТПРА...",negative,636
83467,КУРТКА ПОНРАВИЛАСЬ...ДЖИНСА ДОБРОТНАЯ..ОЖИДАЛА...,positive,839


In [ ]:
tokenizer